In [2]:
import os
import sys
import shutil
import numpy as np
import pandas as pd
#import pydicom as dicom
import matplotlib.pylab as plt
#import SimpleITK as sitk
from datetime import datetime


In [3]:
%env SITK_SHOW_COMMAND "/home/andres/Downloads/Slicer-4.10.2-linux-amd64/Slicer"
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
#dicom.config.convert_wrong_length_to_UN = True

env: SITK_SHOW_COMMAND="/home/andres/Downloads/Slicer-4.10.2-linux-amd64/Slicer"


In [4]:
parent_dir = os.path.abspath('..')
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

from Utils import utils

In [5]:
config = utils.read_config()
config

{'common': {'paths': {'source_path_wd': 'E:/U-CAN-Lymfom_A',
   'source_path_bd': '/media/andres/T7 Shield/ucan_lymfom'}},
 'selection': {'filenames': {'rejected_folder_filename': 'Rejected_exams_from_U-CAN-Lymfom.xlsx',
   'sourcefiltered_folder_filename': 'SourceFiltered_exams_from_U-CAN-Lymfom.xlsx',
   'incomplete_folders_filename_initial': 'Excel_files/06_11_2023/No_PT_or_CT_exams_from_U-CAN-Lymfom_before_selection_process.xlsx',
   'incomplete_folders_filename_final': 'Excel_files/06_11_2023/No_PT_or_CT_exams_from_U-CAN-Lymfom_after_selection_process.xlsx',
   'selected_folders_beforefiltering_filename': 'Excel_files/06_11_2023/Selected_exams_after_filtering_from_U-CAN-Lymfom.xlsx',
   'selected_folders_afterfiltering_filename': 'Excel_files/06_11_2023/Selected_exams_before_filtering_from_U-CAN-Lymfom.xlsx',
   'final_selected_images_filename': 'Excel_files/06_11_2023/Final_Selected_exams_from_U-CAN-Lymfom.xlsx',
   'list_of_distorted_images_filename': 'Excel_files/06_11_2023/exa

In [6]:
#source_path_wd = "/media/andres/T7 Shield1/U-CAN-Lymfom_A/Selected_for_UCAN_project"
source_path_bd = "/media/andres/T7 Shield/ucan_lymfom"

source_path_wd = config['common']['paths']['source_path_wd']
#source_path_bd = config['common']['paths']['source_path_bd']

master_data_wd_filename = config['metadata']['filenames']['final_selected_images_filename']
master_data_bd_filename = config['metadata']['filenames']['final_selected_images_filename']


In [7]:
final_selected_images_filename = config['selection']['filenames']['final_selected_images_filename']
final_selected_folders_bd = source_path_bd + '/' + final_selected_images_filename

In [8]:
# master_data for black disk
print("Reading master data for black disk")
#master_data_bd = pd.read_excel(os.path.join(source_path_bd, master_data_wd_filename))
#master_data_bd = pd.read_excel(source_path_bd + "/Excel_files/" + master_data_wd_filename, parse_dates=["scan_date"])
master_data_bd = pd.read_excel("/media/workstation/T7 Shield/ucan_lymfom/Excel_files/Finalized_dataset.xlsx", parse_dates=["scan_date"])
#master_data_wd[['system', 'npr', 'scan_date']]= master_data_wd['patient_directory'].str.split(pat='_|-', n=2, expand=True)
print(master_data_bd.shape)

master_data_bd.head(2)

Reading master data for black disk


FileNotFoundError: [Errno 2] No such file or directory: '/media/workstation/T7 Shield/ucan_lymfom/Excel_files/Finalized_dataset.xlsx'

In [59]:
#(2264 / 25) * 2 -> 181.12 Gb

In [61]:
#misclassified_df = pd.read_excel(source_path_bd + '/MisclassifiedSelected_files.xlsx')
#misclassified_df = pd.read_excel(source_path_bd + '//Excel_files/MisclassifiedSelected_files.xlsx')
misclassified_df = pd.read_excel("/media/workstation/T7 Shield/ucan_lymfom/Excel_files/MisclassifiedSelected_files.xlsx")
misclassified_df[['source_directory', 'patient_directory', 'PET-CT_info']] = misclassified_df['directory'].str.rsplit(pat='/', n=2, expand=True)
misclassified_patient_directory_list = misclassified_df.patient_directory.to_list()
misclassified_df.head()

,Unnamed: 0,directory,count,source_directory,patient_directory,PET-CT_info
0,284,F:/ucan_lymfom/ASPTCTX0001_npr196644347217-20161125/CT-20161125-114603-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.958984mm,1,F:/ucan_lymfom,ASPTCTX0001_npr196644347217-20161125,CT-20161125-114603-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.958984mm
1,750,F:/ucan_lymfom/ASPTCTX0001_npr338511978351-20170201/CT-20170201-122546-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.953125mm,1,F:/ucan_lymfom,ASPTCTX0001_npr338511978351-20170201,CT-20170201-122546-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.953125mm
2,1398,F:/ucan_lymfom/ASPTCTX0001_npr567278394863-20151103/CT-20151103-130218-6.3_UAS-WB-FDG-3D_CT_THORAX-BUK_VENFAS_20141209-HELKROPP_VENFAS_AX_1MM-0.976562mm,1,F:/ucan_lymfom,ASPTCTX0001_npr567278394863-20151103,CT-20151103-130218-6.3_UAS-WB-FDG-3D_CT_THORAX-BUK_VENFAS_20141209-HELKROPP_VENFAS_AX_1MM-0.976562mm
3,1412,F:/ucan_lymfom/ASPTCTX0001_npr574436824692-20170222/CT-20170222-104855-6.6_UAS-WB-FDG-3D_VENFASTHORAX_INSP_20160120-WB_VEN_AX-0.951172mm,1,F:/ucan_lymfom,ASPTCTX0001_npr574436824692-20170222,CT-20170222-104855-6.6_UAS-WB-FDG-3D_VENFASTHORAX_INSP_20160120-WB_VEN_AX-0.951172mm
4,1682,F:/ucan_lymfom/ASPTCTX0001_npr676639996732-20180928/CT-20180928-150052-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.976562mm,1,F:/ucan_lymfom,ASPTCTX0001_npr676639996732-20180928,CT-20180928-150052-6.3_UAS-WB-FDG-3D_CT_VENFAS-WB_VEN_AX-0.976562mm


In [62]:
master_data_bd[np.logical_and(master_data_bd.num_slices >200,master_data_bd.num_slices < 250)].head(2) # 488

,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,2096,F:/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,F:/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,ASPTCTX0001,lpr385705046400,2014-03-13,13743,1.2.840.113619.2.55.3.4240671514.332.1394175950.81.1,lpr385705046400,33.0,'69.0','1.73',512,512,215,0.976562,0.976562,3.75,"(512, 512, 215)","(0.976562, 0.976562, 3.75)","[-250.000, -250.000, 167.080]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20140313','152608','FDG WB 1 h','STANDARD CT RECON','6.1 UAS-WB-FDG-3D',NaN,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2095,F:/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,F:/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,ASPTCTX0001,lpr385705046400,2014-03-13,13745,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,33.0,'69.0','1.73',128,128,247,3.906250,3.906250,3.27,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 169.020004272460]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20140313','154238','FDG WB 1 h','UAS-WB-FDG-3D-AC','UAS-WB-FDG-3D',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'


In [63]:
exams_with_lt250 = master_data_bd[master_data_bd.num_slices < 250].patient_directory.to_list()
len(exams_with_lt250)

499

In [64]:
master_data_bd_filtered = master_data_bd[~np.logical_or(master_data_bd.patient_directory.isin(exams_with_lt250), master_data_bd.patient_directory.isin(misclassified_patient_directory_list))].copy().reset_index()
master_data_bd_filtered.shape

(1560, 46)

In [65]:
master_data_bd_filtered.patient_directory.nunique()

780

In [66]:
master_data_bd_filtered.patient_directory.nunique()

780

In [67]:
config_size = config['resampling']['image_size']
config_spacing = config['resampling']['voxel_spacing']
print('config_size: ', config_size)
print('config_spacing: ', config_spacing)

config_size:  [256, 256, 256]
config_spacing:  [2.0364201068878174, 2.0364201068878174, 3.0]


In [69]:
#clip positive values at +3000
#set outside image 0 values to -1024

In [70]:
master_data_bd_filtered.head(2)

,index,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,2,2,948,F:/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,F:/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,lpr415675513429,2019-02-01,14250,1.2.840.113619.2.5.91447938.12625.1549010475.262,lpr415675513429,32.0,'90.0','1.8',512,512,382,0.962891,0.962891,3.00,"(512, 512, 382)","(0.962891, 0.962891, 3.0)","[-249.237, -234.463, -127.315]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20190201','102917.254016','PET FDG WB 1 h','WB Venfas 3 ax','3.5 WB_FDG CT VENFAS',NaN,CT,'GE MEDICAL SYSTEMS','Discovery MI',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,3,947,F:/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,F:/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,ASPTCTX0001,lpr415675513429,2019-02-01,14246,1.2.840.113619.2.405.3842663696.1549013394.986841,lpr415675513429,32.0,'90.0','1.8',256,256,341,1.953125,1.953125,2.79,"(256, 256, 341)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, -128.24998474121]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'QCFX',"['ORIGINAL', 'PRIMARY']",'20190201','102509','PET FDG WB 1 h','QCFX-S 400','3.5 WB_FDG CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RANSNG', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery MI','FDG -- fluorodeoxyglucose','0.0','090500.00','264312992.0','6586.2001953125','0.96700000762939','20190201090500.00'


In [71]:
master_data_bd_filtered = master_data_bd_filtered.sort_values(by=["patient_directory"	,"PET-CT_info"], ascending=[True, False])
master_data_bd_filtered.head(2)

,index,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
1,3,3,947,F:/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,F:/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,ASPTCTX0001,lpr415675513429,2019-02-01,14246,1.2.840.113619.2.405.3842663696.1549013394.986841,lpr415675513429,32.0,'90.0','1.8',256,256,341,1.953125,1.953125,2.79,"(256, 256, 341)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, -128.24998474121]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'QCFX',"['ORIGINAL', 'PRIMARY']",'20190201','102509','PET FDG WB 1 h','QCFX-S 400','3.5 WB_FDG CT VENFAS',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RANSNG', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery MI','FDG -- fluorodeoxyglucose','0.0','090500.00','264312992.0','6586.2001953125','0.96700000762939','20190201090500.00'
0,2,2,948,F:/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,F:/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,lpr415675513429,2019-02-01,14250,1.2.840.113619.2.5.91447938.12625.1549010475.262,lpr415675513429,32.0,'90.0','1.8',512,512,382,0.962891,0.962891,3.00,"(512, 512, 382)","(0.962891, 0.962891, 3.0)","[-249.237, -234.463, -127.315]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20190201','102917.254016','PET FDG WB 1 h','WB Venfas 3 ax','3.5 WB_FDG CT VENFAS',NaN,CT,'GE MEDICAL SYSTEMS','Discovery MI',NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Selected exams and clinical data matching

In [203]:
#selectedexams_path = "/home/workstation/Ashish/UCAN/Final_Selected_exams_from_U-CAN-Lymfom.xlsx"
selectedexams_path = "/home/ashish/Ashish/UCAN/06_11_2023/Finalized_dataset.xlsx"
#clinicaldata_path = "/home/workstation/Ashish/UCAN/clinical_data.xlsx"
#clinicaldata_path = "/home/workstation/Ashish/UCAN/U-CAN Lymfom - Ahlström 211021 - utlämnat_pnr_removed.xlsx"
#clinicaldata_path = "F:/ucan_lymfom/ClinicalData/U-CAN Lymfom - Ahlström 211021 - utlämnat_pnr_removed.xlsx"

#clinicaldata_path = "/home/ashish/Ashish/UCAN/U-CAN Lymfom - Ahlström 211021 - utlämnat_pnr_removed.xlsx"
clinicaldata_path = "/home/ashish/Ashish/UCAN/20231129_UCAN_lym_export/Lymphoma_Octopus_Export 2023-11-29_C80_only_n1291_incl_YYYY_MM_pnr_removed.xlsx"

In [204]:
master_data = pd.read_excel(selectedexams_path)
print(master_data.shape)
master_data.head()

(3646, 45)


,Unnamed: 0,Unnamed: 0_x,directory,source_directory,patient_directory,PET-CT_info,system,npr,scan_date,Unnamed: 0_y,dicom_img,patient_id,patient_age,patient_weight,patient_size,imgsz_x,imgsz_y,num_slices,voxsz_x,voxsz_y,slice_thickness,image_size,voxel_size,img_pos,img_orient,for_uid,att_corr,recons_method,image_type,aquisition_dt,aquisition_time,study_desc,series_desc,protocol,corr_img,modality,manufacturer,manufacturer_model,radiopharmaceutical,radiopharmaceutical_volume,radiopharmaceutical_start_time,radionuclide_total_dose,radionuclide_half_life,radionuclide_positron_fraction,radiopharmaceutical_start_date_time
0,0,3689,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,CT-20140313-152551-6.1_UAS-WB-FDG-3D-STANDARD_CT_RECON-3.750000mm,ASPTCTX0001,lpr385705046400,20140313,13743,1.2.840.113619.2.55.3.4240671514.332.1394175950.81.1,lpr385705046400,33.0,'69.0','1.73',512,512,215,0.976562,0.976562,3.750000,"(512, 512, 215)","(0.976562, 0.976562, 3.75)","[-250.000, -250.000, 167.080]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,NaN,NaN,"['ORIGINAL', 'PRIMARY', 'AXIAL']",'20140313','152608','FDG WB 1 h','STANDARD CT RECON','6.1 UAS-WB-FDG-3D',NaN,CT,'GE MEDICAL SYSTEMS','Discovery STE',NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,3690,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr385705046400-20140313/PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr385705046400-20140313,PT-20140313-152728-UAS-WB-FDG-3D-UAS-WB-FDG-3D-AC-3.2700mm,ASPTCTX0001,lpr385705046400,20140313,13745,1.2.840.113619.2.131.4240671514.1394722651.463822,lpr385705046400,33.0,'69.0','1.73',128,128,247,3.906250,3.906250,3.270000,"(128, 128, 247)","(3.90625, 3.90625, 3.27)","[-248.04687500000, -248.04687500000, 169.020004272460]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.55.3.4240671514.332.1394175949.830.13830.1,"'measured,, 0.096000 cm-1,'",'3D IR',"['ORIGINAL', 'PRIMARY']",'20140313','154238','FDG WB 1 h','UAS-WB-FDG-3D-AC','UAS-WB-FDG-3D',"['DECY', 'ATTN', 'SCAT', 'DTIM', 'RAN', 'DCAL', 'SLSENS', 'NORM']",PT,'GE MEDICAL SYSTEMS','Discovery STE','FDG -- fluorodeoxyglucose','0.0','141400.00','259344912.0','6588.0','0.97000002861023','20140313141400.00'
2,2,3687,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,CT-20190201-103111-3.5_WB_FDG_CT_VENFAS-WB_Venfas_3_ax-3.000000mm,ASPTCTX0001,lpr415675513429,20190201,14250,1.2.840.113619.2.5.91447938.12625.1549010475.262,lpr415675513429,32.0,'90.0','1.8',512,512,382,0.962891,0.962891,3.000000,"(512, 512, 382)","(0.962891, 0.962891, 3.0)","[-249.237, -234.463, -127.315]","[1.000000, 0.000000, 0.000000, 0.000000, 1.000000, 0.000000]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,NaN,NaN,"['DERIVED', 'SECONDARY', 'REFORMATTED', 'AVERAGE']",'20190201','102917.254016','PET FDG WB 1 h','WB Venfas 3 ax','3.5 WB_FDG CT VENFAS',NaN,CT,'GE MEDICAL SYSTEMS','Discovery MI',NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,3688,/media/andres/T7 Shield/ucan_lymfom/ASPTCTX0001_lpr415675513429-20190201/PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,/media/andres/T7 Shield/ucan_lymfom,ASPTCTX0001_lpr415675513429-20190201,PT-20190201-101500-3.5_WB_FDG_CT_VENFAS-QCFX-S_400-2.7900mm,ASPTCTX0001,lpr415675513429,20190201,14246,1.2.840.113619.2.405.3842663696.1549013394.986841,lpr415675513429,32.0,'90.0','1.8',256,256,341,1.953125,1.953125,2.790000,"(256, 256, 341)","(1.953125, 1.953125, 2.79)","[-249.02343750000, -249.02343750000, -128.24998474121]","[1, 0, 0, 0, 1, -0]",1.2.840.113619.2.405.3.3842663696.892.1549005827.841.4795.1,"'measured,, 0.000000 cm-1, attenuation smooth'",'QCFX',"['OR

In [205]:
#exams_data = master_data_bd_filtered[["patient_directory", "system", "npr", "scan_date"]].copy().sort_values(by=["scan_date"]).reset_index(drop=True)
#exams_data = master_data[["patient_directory", "system", "npr", "scan_date"]].copy().sort_values(by=["scan_date"]).reset_index(drop=True)
exams_data = master_data[["patient_directory", "system", "npr", "scan_date"]].copy().sort_values(by=["npr", "scan_date"]).reset_index(drop=True)

exams_data["scan_date"] = pd.to_datetime(exams_data["scan_date"], format="%Y%m%d")
exams_data = exams_data.dropna().reset_index(drop=True)
exams_data = exams_data.drop_duplicates().reset_index(drop=True)
#exams_data.scan_date = exams_data.scan_date.astype('datetime64[ns]')
exams_data.npr = exams_data.npr.str.replace(" ","")
print(exams_data.shape)
exams_data.head(2)

(1823, 4)


,patient_directory,system,npr,scan_date
0,ASPTCTX0001_lpr385705046400-20140313,ASPTCTX0001,lpr385705046400,2014-03-13
1,ASPTCTX0001_lpr415675513429-20190201,ASPTCTX0001,lpr415675513429,2019-02-01


In [207]:
exams_data.dtypes

patient_directory            object
system                       object
npr                          object
scan_date            datetime64[ns]
dtype: object

In [208]:
#clinical_data = pd.read_excel(clinicaldata_path).sort_values(by=["consentDate"]).reset_index(drop=True)
#clinical_data = pd.read_excel(clinicaldata_path, header=1).sort_values(by=["personReference", "consentDate"]).reset_index(drop=True)
clinical_data = pd.read_excel(clinicaldata_path).sort_values(by=["personReference", "consentDate"]).reset_index(drop=True)
clinical_data.columns = clinical_data.columns.str.strip()
clinical_data.npr = clinical_data.personReference.str.replace(" ","")
print(clinical_data.shape)
clinical_data.head(2)

(1291, 44)


/tmp/ipykernel_18517/3464410697.py:5: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  clinical_data.npr = clinical_data.personReference.str.replace(" ","")


,personReference,BirthYYYYMM,BirthYear,BirthMonth,inclusionYear,includingHospital,county,consentDate,consentComments,generalComments,deceased,deceasedDate,sex,diagnosisGroups,DiaDate,stadium,translatedDiagnosis,diagnosisComments,DiaDate 2,translatedDiagnosis 2,diagnosisComments 2,type,indication,startDate,endDate,typeDetails,OP/BIOPSYdate,type.1,tissueType,tissueTypeOther,surgeryComments,OP/BIOPSYdate 2,type 2,tissueType 2,tissueTypeOther 2,surgeryComments 2,RELAPSE1,date,verifictationType,relapseComments,RELAPSE2,date 2,verifictationType 2,relapseComments 2
0,lpr385705046400,198011,1980,11,2014.0,UPPSALA,UPPSALA,2014-03-18,Javier Perez Quinones. Namn finns inte i folkbokföringen varför inte står ovan?,"Prov finns 2014-03-19, UCAN 1 + 2014-05-27, UCAN 2 ""mittprov"". Emigrerat åter till Kuba juni 2014. Gästforskare UU vid diagnos och behandling + inklusion U-CAN. Extern RT planerad ges på Kuba.",False,NaN,UNKNOWN,HEMA,2014-03-17,AASTADIUM2,"C81.9 - Hodgkins lymfom, ospecificerat",2A,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2014-03-19,2014-06-26,ABVD,2014-03-11,BIOPSY,OTHER,Mellannålsbiopsier lymfkörtlar,HL,2014-03-17,BIOPSY,OTHER,Lymfkörtel höger fossa supraclav,HL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,lpr415675513429,198602,1986,2,2019.0,UPPSALA,FOREIGN,2019-02-13,NaN,Mats Liljeros. Åland,False,NaN,UNKNOWN,HEMA,2019-02-12,AASTADIUM4,"C81.9 - Hodgkins lymfom, ospecificerat",NaN,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2019-02-13,2019-05-26,BEACOPP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
clinical_data.personReference.nunique()

1291

### Match based on npr

In [211]:
linked_data = pd.merge(exams_data, clinical_data, how="inner", left_on=["npr"], right_on=["personReference"], suffixes=['','_'])
print(linked_data.shape)

linked_data.head(2)

(738, 48)


,patient_directory,system,npr,scan_date,personReference,BirthYYYYMM,BirthYear,BirthMonth,inclusionYear,includingHospital,county,consentDate,consentComments,generalComments,deceased,deceasedDate,sex,diagnosisGroups,DiaDate,stadium,translatedDiagnosis,diagnosisComments,DiaDate 2,translatedDiagnosis 2,diagnosisComments 2,type,indication,startDate,endDate,typeDetails,OP/BIOPSYdate,type.1,tissueType,tissueTypeOther,surgeryComments,OP/BIOPSYdate 2,type 2,tissueType 2,tissueTypeOther 2,surgeryComments 2,RELAPSE1,date,verifictationType,relapseComments,RELAPSE2,date 2,verifictationType 2,relapseComments 2
0,ASPTCTX0001_lpr385705046400-20140313,ASPTCTX0001,lpr385705046400,2014-03-13,lpr385705046400,198011,1980,11,2014.0,UPPSALA,UPPSALA,2014-03-18,Javier Perez Quinones. Namn finns inte i folkbokföringen varför inte står ovan?,"Prov finns 2014-03-19, UCAN 1 + 2014-05-27, UCAN 2 ""mittprov"". Emigrerat åter till Kuba juni 2014. Gästforskare UU vid diagnos och behandling + inklusion U-CAN. Extern RT planerad ges på Kuba.",False,NaN,UNKNOWN,HEMA,2014-03-17,AASTADIUM2,"C81.9 - Hodgkins lymfom, ospecificerat",2A,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2014-03-19,2014-06-26,ABVD,2014-03-11,BIOPSY,OTHER,Mellannålsbiopsier lymfkörtlar,HL,2014-03-17,BIOPSY,OTHER,Lymfkörtel höger fossa supraclav,HL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ASPTCTX0001_lpr415675513429-20190201,ASPTCTX0001,lpr415675513429,2019-02-01,lpr415675513429,198602,1986,2,2019.0,UPPSALA,FOREIGN,2019-02-13,NaN,Mats Liljeros. Åland,False,NaN,UNKNOWN,HEMA,2019-02-12,AASTADIUM4,"C81.9 - Hodgkins lymfom, ospecificerat",NaN,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2019-02-13,2019-05-26,BEACOPP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [212]:
linked_data.npr.nunique()

371

In [213]:
#linked_data.to_excel('/home/ashish/Ashish/UCAN/linked_data_390patientmatches_clinicaldata_export_211021.xlsx', index=False)
linked_data.to_excel('/home/ashish/Ashish/UCAN/linked_data_371patientmatches_clinicaldata_export_20231129.xlsx', index=False)

### Match based on ranking

In [ ]:
#exams_data["rank"] = exams_data.groupby("npr")["scan_date"].rank(method="first", ascending=True)
#utils.display_full(exams_data)

In [ ]:
#clinical_data["rank"] = clinical_data.groupby("personReference")["consentDate"].rank(method="first", ascending=True)
#utils.display_full(clinical_data)

In [86]:
linked_data2 = pd.merge(exams_data, clinical_data, how="inner", left_on=["npr", "rank"], right_on=["personReference", "rank"], suffixes=['','_'])
print(linked_data2.shape)
linked_data2.to_excel('/home/ashish/Ashish/UCAN/linked_data_387matches_27nov2023.xlsx', index=False)
linked_data2.head(2)

KeyError: 'rank'

In [27]:
linked_data2["npr"].nunique()

387

In [43]:
linked_data["includingHospital"].value_counts()

includingHospital
UPPSALA     128
FALUN        58
GAVLE        44
KARLSTAD      6
VASTERAS      1
Name: count, dtype: int64

In [44]:
linked_data["county"].value_counts()

county
UPPSALA           213
DALARNA            74
GAVLEBORG          56
SODERMANLAND       14
VASTMANLAND        13
VARMLAND            4
STOCKHOLM           3
FOREIGN             2
OSTERGOTLAND        2
VASTERNORRLAND      1
Name: count, dtype: int64

In [29]:
linked_data["sex"].value_counts()

sex
MALE       234
FEMALE     151
UNKNOWN      2
Name: count, dtype: int64

In [32]:
linked_data["diagnosisGroups"].value_counts()

diagnosisGroups
HEMA               334
HEMA, CTRL          16
CTRL, HEMA          10
HEMA, LC             4
CTRL, HEMA, PAT      3
HEMA, PAT            3
NEURO, HEMA          2
CRC, HEMA            2
HEMA, NEURO          2
HEMA, GYN            2
LC, CTRL, HEMA       1
HEMA, PRC            1
PAT, HEMA            1
HEMA, CRC            1
PRC, HEMA            1
LC, HEMA             1
GYN, LC, HEMA        1
CTRL, PAT, HEMA      1
LC, CRC, HEMA        1
Name: count, dtype: int64

In [31]:
linked_data["Diagnosis"].value_counts()

Diagnosis
C83.3 Diffust storcelligt B-cellslymfom                                                          87
C81.1 Hodgkins lymfom (klassiskt) med nodulär skleros                                            51
C81.9 Hodgkins lymfom, ospecificerat                                                             50
C85.1B B-cellslymfom, högmalignt, ospecificerat                                                  33
C83.1 Mantelcellslymfom                                                                          27
C91.1 Kronisk lymfatisk leukemi av Bcellstyp [B-KLL]                                             19
C82.9 Follikulärt lymfom, ospecificerat                                                          13
C85.1 B-cellslymfom, ospecificerat                                                               12
C81.2 Hodgkins lymfom (klassiskt) med blandad cellularitet                                       10
C88.4 Extranodalt marginalzons-B-cellslymfom av mukosaassocierad lymfoid vävnad [MALT-lymf

### Matching based on closest date

In [75]:
clinical_data.consentDate.unique()

array(['2010-04-11', '2010-06-16', '2010-07-02', ..., '2021-10-06',
       '2021-10-11', nan], dtype=object)

In [144]:
clinical_data_test = clinical_data[["personReference", "consentDate", "sex", "diagnosisGroups"]].copy().reset_index(drop=True)
print(clinical_data_test.shape)
#clinical_data_test.columns = [["personReference", "consentDate"]]
print(clinical_data_test.personReference.nunique())
clinical_data_test = clinical_data_test.dropna().reset_index(drop=True)
print(clinical_data_test.shape)
clinical_data_test = clinical_data_test.drop_duplicates().reset_index(drop=True)
print(clinical_data_test.shape)
clinical_data_test["personReference"] = clinical_data_test["personReference"].astype('object')
clinical_data_test["consentDate"] = clinical_data_test["consentDate"].astype('datetime64[ns]')
print(clinical_data_test.shape)
clinical_data_test.head(2)

(1480, 4)
1480
(1468, 4)
(1468, 4)
(1468, 4)


,personReference,consentDate,sex,diagnosisGroups
0,npr446854059583,2010-04-11,MALE,HEMA
1,npr165959402726,2010-06-16,MALE,HEMA


In [77]:
print(exams_data.shape)
exams_data = exams_data.drop_duplicates()
print(exams_data.shape)

(780, 4)
(780, 4)


In [78]:
clinical_data_test.dtypes

personReference            object
consentDate        datetime64[ns]
sex                        object
diagnosisGroups            object
dtype: object

In [79]:
clinical_data_test.consentDate

0      2010-04-11
1      2010-06-16
2      2010-07-02
3      2010-09-13
4      2010-09-16
          ...    
1463   2021-09-30
1464   2021-10-06
1465   2021-10-06
1466   2021-10-11
1467   2021-10-11
Name: consentDate, Length: 1468, dtype: datetime64[ns]

In [80]:
clinical_data_test.consentDate.dtype

dtype('<M8[ns]')

In [81]:
exams_data.scan_date.dtype

dtype('<M8[ns]')

In [82]:
clinical_data_test.consentDate.values.dtype

dtype('<M8[ns]')

In [83]:
utils.display_full(clinical_data_test)

      personReference consentDate      sex       diagnosisGroups
0     npr446854059583  2010-04-11     MALE                  HEMA
1     npr165959402726  2010-06-16     MALE                  HEMA
2     npr844745391915  2010-07-02     MALE              LC, HEMA
3     npr351786103610  2010-09-13     MALE                  HEMA
4     npr253202446506  2010-09-16     MALE                  HEMA
5     npr671162790654  2010-09-22     MALE             CRC, HEMA
6     npr685106741724  2010-09-22   FEMALE                  HEMA
7     npr728798676693  2010-10-05     MALE                  HEMA
8     npr824033832482  2010-10-08   FEMALE                  HEMA
9     npr972308330752  2010-10-12   FEMALE                  HEMA
10    npr411742927511  2010-10-12     MALE                  HEMA
11    npr694994026946  2010-10-12     MALE                  HEMA
12    npr548114351603  2010-10-22     MALE                  HEMA
13    npr491098418481  2010-10-25     MALE                  HEMA
14    npr330543079169  20

In [84]:
exams_data.dtypes

patient_directory            object
system                       object
npr                          object
scan_date            datetime64[ns]
dtype: object

In [85]:
exams_data.scan_date.dtype

dtype('<M8[ns]')

In [86]:
linked_data = pd.merge(exams_data, clinical_data_test, how="inner", left_on=["npr", "scan_date"], right_on=["personReference", "consentDate"], suffixes=['','_'])
linked_data

,patient_directory,system,npr,scan_date,personReference,consentDate,sex,diagnosisGroups
0,ASPTCTX0001_npr351786103610-20100913,ASPTCTX0001,npr351786103610,2010-09-13,npr351786103610,2010-09-13,MALE,HEMA
1,ASPTCTX0001_npr481526034495-20141106,ASPTCTX0001,npr481526034495,2014-11-06,npr481526034495,2014-11-06,MALE,HEMA
2,ASPTCTX0001_npr209149612467-20150706,ASPTCTX0001,npr209149612467,2015-07-06,npr209149612467,2015-07-06,FEMALE,HEMA
3,ASPTCTX0001_npr216618605686-20180103,ASPTCTX0001,npr216618605686,2018-01-03,npr216618605686,2018-01-03,FEMALE,HEMA
4,ASPTCTX0001_npr261871834244-20180706,ASPTCTX0001,npr261871834244,2018-07-06,npr261871834244,2018-07-06,MALE,HEMA
5,ASPTCTX0001_npr724286675861-20190410,ASPTCTX0001,npr724286675861,2019-04-10,npr724286675861,2019-04-10,MALE,HEMA
6,ASPTCTX0001_npr750134936705-20190529,ASPTCTX0001,npr750134936705,2019-05-29,npr750134936705,2019-05-29,MALE,HEMA
7,ASPTCTX0001_npr453276486712-20201126,ASPTCTX0001,npr453276486712,2020-11-26,npr453276486712,2020-11-26,FEMALE,HEMA


In [87]:
linked_data.shape

(8, 8)

In [88]:
print(exams_data.shape)
print(clinical_data_test.shape)

(780, 4)
(1468, 4)


In [135]:
linked_data = pd.merge_asof(exams_data, clinical_data_test, left_on="scan_date", right_on="consentDate", left_by="npr", right_by="personReference",  tolerance=pd.Timedelta("365d"), direction="forward")
print(linked_data.shape)
linked_data.head()

(780, 8)


,patient_directory,system,npr,scan_date,personReference,consentDate,sex,diagnosisGroups
0,ASPTCTX0001_npr967652562138-20100730,ASPTCTX0001,npr967652562138,2010-07-30,npr967652562138,2011-06-22,MALE,HEMA
1,ASPTCTX0001_npr355725597861-20100816,ASPTCTX0001,npr355725597861,2010-08-16,NaN,NaT,NaN,NaN
2,ASPTCTX0001_npr351786103610-20100913,ASPTCTX0001,npr351786103610,2010-09-13,npr351786103610,2010-09-13,MALE,HEMA
3,ASPTCTX0001_npr967652562138-20100930,ASPTCTX0001,npr967652562138,2010-09-30,npr967652562138,2011-06-22,MALE,HEMA
4,ASPTCTX0001_npr102928239238-20101115,ASPTCTX0001,npr102928239238,2010-11-15,NaN,NaT,NaN,NaN


In [136]:
linked_data.npr.nunique()

550

In [132]:
exams_data[exams_data.npr=='npr967652562138']

,patient_directory,system,npr,scan_date
0,ASPTCTX0001_npr967652562138-20100730,ASPTCTX0001,npr967652562138,2010-07-30
3,ASPTCTX0001_npr967652562138-20100930,ASPTCTX0001,npr967652562138,2010-09-30
9,ASPTCTX0001_npr967652562138-20110527,ASPTCTX0001,npr967652562138,2011-05-27
16,ASPTCTX0001_npr967652562138-20111005,ASPTCTX0001,npr967652562138,2011-10-05
18,ASPTCTX0001_npr967652562138-20120206,ASPTCTX0001,npr967652562138,2012-02-06
22,ASPTCTX0001_npr967652562138-20120321,ASPTCTX0001,npr967652562138,2012-03-21


In [131]:
utils.display_full(linked_data[~linked_data.npr.isin(linked_data2_npr_list)])

                        patient_directory       system              npr  scan_date  personReference consentDate   sex diagnosisGroups
0    ASPTCTX0001_npr967652562138-20100730  ASPTCTX0001  npr967652562138 2010-07-30  npr967652562138  2011-06-22  MALE            HEMA
1    ASPTCTX0001_npr355725597861-20100816  ASPTCTX0001  npr355725597861 2010-08-16              NaN         NaT   NaN             NaN
3    ASPTCTX0001_npr967652562138-20100930  ASPTCTX0001  npr967652562138 2010-09-30  npr967652562138  2011-06-22  MALE            HEMA
4    ASPTCTX0001_npr102928239238-20101115  ASPTCTX0001  npr102928239238 2010-11-15              NaN         NaT   NaN             NaN
9    ASPTCTX0001_npr967652562138-20110527  ASPTCTX0001  npr967652562138 2011-05-27  npr967652562138  2011-06-22  MALE            HEMA
10   ASPTCTX0001_npr113096472044-20110531  ASPTCTX0001  npr113096472044 2011-05-31              NaN         NaT   NaN             NaN
11   ASPTCTX0001_npr987467931634-20110610  ASPTCTX0001  npr987

In [129]:
linked_data[~linked_data.npr.isin(linked_data2_npr_list)].npr.nunique()

472

In [137]:
linked_data1 = linked_data.dropna().reset_index(drop=True).copy()
print(linked_data1.shape)
#(1777, 8)

(81, 8)


In [138]:
linked_data1 = linked_data1.sort_values(by=["npr", "scan_date"]).reset_index(drop=True)
print(linked_data1.shape)
utils.display_full(linked_data1)

(81, 8)
                       patient_directory       system              npr  scan_date  personReference consentDate      sex  diagnosisGroups
0   ASPTCTX0001_lpr415675513429-20190201  ASPTCTX0001  lpr415675513429 2019-02-01  lpr415675513429  2019-02-13  UNKNOWN             HEMA
1   ASPTCTX0001_npr100169878450-20130412  ASPTCTX0001  npr100169878450 2013-04-12  npr100169878450  2013-04-17     MALE             HEMA
2   ASPTCTX0001_npr107605794128-20210520  ASPTCTX0001  npr107605794128 2021-05-20  npr107605794128  2021-05-26     MALE             HEMA
3   ASPTCTX0001_npr116141176598-20191025  ASPTCTX0001  npr116141176598 2019-10-25  npr116141176598  2019-11-05   FEMALE       HEMA, CTRL
4   ASPTCTX0001_npr137231127717-20170710  ASPTCTX0001  npr137231127717 2017-07-10  npr137231127717  2017-11-14   FEMALE             HEMA
5   ASPTCTX0001_npr151152435400-20140917  ASPTCTX0001  npr151152435400 2014-09-17  npr151152435400  2014-12-04     MALE             HEMA
6   ASPTCTX0001_npr153663821130-2

In [102]:
linked_data1.npr.nunique()

79

In [139]:
multiple_scans = linked_data1.groupby('npr').agg({'scan_date':'count'}).reset_index()
multiple_scans = multiple_scans[multiple_scans.scan_date>1]
multiple_scans

,npr,scan_date
76,npr967652562138,3


In [140]:
linked_data1[linked_data1.npr=='npr967652562138']

,patient_directory,system,npr,scan_date,personReference,consentDate,sex,diagnosisGroups
76,ASPTCTX0001_npr967652562138-20100730,ASPTCTX0001,npr967652562138,2010-07-30,npr967652562138,2011-06-22,MALE,HEMA
77,ASPTCTX0001_npr967652562138-20100930,ASPTCTX0001,npr967652562138,2010-09-30,npr967652562138,2011-06-22,MALE,HEMA
78,ASPTCTX0001_npr967652562138-20110527,ASPTCTX0001,npr967652562138,2011-05-27,npr967652562138,2011-06-22,MALE,HEMA


In [141]:
linked_data2 = linked_data1[~linked_data1.npr.isin(multiple_scans.npr.to_list())].reset_index(drop=True).copy()
linked_data2_npr_list = linked_data2.npr.to_list()
utils.display_full(linked_data2)

                       patient_directory       system              npr  scan_date  personReference consentDate      sex  diagnosisGroups
0   ASPTCTX0001_lpr415675513429-20190201  ASPTCTX0001  lpr415675513429 2019-02-01  lpr415675513429  2019-02-13  UNKNOWN             HEMA
1   ASPTCTX0001_npr100169878450-20130412  ASPTCTX0001  npr100169878450 2013-04-12  npr100169878450  2013-04-17     MALE             HEMA
2   ASPTCTX0001_npr107605794128-20210520  ASPTCTX0001  npr107605794128 2021-05-20  npr107605794128  2021-05-26     MALE             HEMA
3   ASPTCTX0001_npr116141176598-20191025  ASPTCTX0001  npr116141176598 2019-10-25  npr116141176598  2019-11-05   FEMALE       HEMA, CTRL
4   ASPTCTX0001_npr137231127717-20170710  ASPTCTX0001  npr137231127717 2017-07-10  npr137231127717  2017-11-14   FEMALE             HEMA
5   ASPTCTX0001_npr151152435400-20140917  ASPTCTX0001  npr151152435400 2014-09-17  npr151152435400  2014-12-04     MALE             HEMA
6   ASPTCTX0001_npr153663821130-20210505 

In [142]:
linked_data2.to_excel('/home/workstation/Ashish/UCAN/linked_data_780_6Nov.xlsx', index=False)

In [167]:
exams_npr_list = exams_data.npr.str.replace(" ","").to_list()
print(len(exams_npr_list))
exams_npr_set = set(exams_npr_list)
print(len(exams_npr_set))

1823
1076


In [168]:
len(exams_npr_set)

1076

In [169]:
exams_npr_set

{'npr959703731992',
 'npr926701257290',
 'npr711322681344',
 'npr250672905402',
 'npr467134513124',
 'npr344930616506',
 'npr261612855696',
 'npr167021644105',
 'npr421064151804',
 'npr444859213901',
 'npr971885828856',
 'npr882266785335',
 'npr809610189287',
 'npr437726777984',
 'npr679961010342',
 'npr796901355424',
 'npr216618605686',
 'npr460936929656',
 'npr196288896583',
 'npr227571410839',
 'npr788802379464',
 'npr766758231404',
 'npr217008694744',
 'npr517273193557',
 'npr769123029315',
 'npr929662475345',
 'npr205346582030',
 'npr353131040011',
 'npr981339069173',
 'npr151462634697',
 'npr874401858520',
 'npr291281941375',
 'npr526794682703',
 'npr428707591273',
 'npr323296159264',
 'npr203979873997',
 'npr413511988429',
 'npr720175180360',
 'npr170667868768',
 'npr253690274264',
 'npr456413476486',
 'npr696287093420',
 'npr413803357102',
 'npr886970012808',
 'npr750785342192',
 'npr982307732936',
 'npr274962080070',
 'npr801182059852',
 'npr312833307758',
 'npr671684118786',


In [170]:
clinical_npr_list = clinical_data.personReference.str.replace(" ","").to_list()
print(len(clinical_npr_list))
clinical_npr_set = set(clinical_npr_list)
print(len(clinical_npr_set))

1291
1291


In [171]:
common = exams_npr_set.intersection(clinical_npr_set)
len(common)

371

In [74]:
clinical_data[clinical_data.personReference.isin(exams_npr_list)]

,personReference,BirthYYYYMM,BirthYear,BirthMonth,inclusionYear,includingHospital,county,consentDate,consentComments,generalComments,deceased,deceasedDate,sex,diagnosisGroups,DiaDate,stadium,translatedDiagnosis,diagnosisComments,DiaDate 2,translatedDiagnosis 2,diagnosisComments 2,type,indication,startDate,endDate,typeDetails,OP/BIOPSYdate,type.1,tissueType,tissueTypeOther,surgeryComments,OP/BIOPSYdate 2,type 2,tissueType 2,tissueTypeOther 2,surgeryComments 2,RELAPSE1,date,verifictationType,relapseComments,RELAPSE2,date 2,verifictationType 2,relapseComments 2
0,lpr385705046400,198011,1980,11,2014.0,UPPSALA,UPPSALA,2014-03-18,Javier Perez Quinones. Namn finns inte i folkbokföringen varför inte står ovan?,"Prov finns 2014-03-19, UCAN 1 + 2014-05-27, UCAN 2 ""mittprov"". Emigrerat åter till Kuba juni 2014. Gästforskare UU vid diagnos och behandling + inklusion U-CAN. Extern RT planerad ges på Kuba.",False,NaN,UNKNOWN,HEMA,2014-03-17,AASTADIUM2,"C81.9 - Hodgkins lymfom, ospecificerat",2A,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2014-03-19,2014-06-26,ABVD,2014-03-11,BIOPSY,OTHER,Mellannålsbiopsier lymfkörtlar,HL,2014-03-17,BIOPSY,OTHER,Lymfkörtel höger fossa supraclav,HL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,lpr415675513429,198602,1986,2,2019.0,UPPSALA,FOREIGN,2019-02-13,NaN,Mats Liljeros. Åland,False,NaN,UNKNOWN,HEMA,2019-02-12,AASTADIUM4,"C81.9 - Hodgkins lymfom, ospecificerat",NaN,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2019-02-13,2019-05-26,BEACOPP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,npr100169878450,194412,1944,12,2013.0,UPPSALA,UPPSALA,2013-04-17,NaN,NaN,False,NaN,MALE,HEMA,2013-03-28,NaN,C81.1 - Hodgkins lymfom (klassiskt) med nodulär skleros,NaN,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2013-04-17,2013-09-18,ABVD,2013-03-28,BIOPSY,OTHER,Lymfkörtelbiopsi,Vävnadsprov vid diagnos: HL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,npr106484754818,197209,1972,9,2020.0,UPPSALA,UPPSALA,2020-06-18,Muntligt samtycke från 2020-06-04.,"AMINI II ej taget. Kontroller ONK avslutade oktober 2022, åter vb.",False,NaN,MALE,"HEMA, CTRL",2020-05-06,AASTADIUM1,"C85.1 - B-cellslymfom, ospecificerat",1A. Lymfkörtel hals sinister.,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2020-06-04,2020-08-20,R-CHOP,2020-05-06,BIOPSY,OTHER,Lymfkörtel vänster hals,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,npr107605794128,194612,1946,12,2021.0,GAVLE,GAVLEBORG,2021-05-26,NaN,NaN,False,NaN,MALE,HEMA,2021-04-07,NaN,C83.1 - Mantelcellslymfom,NaN,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2021-07-22,2021-12-10,R-Benda x6,2021-04-07,BIOPSY,OTHER,Bindehinna hö öga,NaN,2021-05-26,BIOPSY,OTHER,vä näskavitet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1278,npr988275348495,194509,1945,9,2021.0,UPPSALA,UPPSALA,2021-10-28,Muntligt samtycke per 2021-09-24.,AVLIDEN 2023-08-18. EJ AMINI-prover. Inkl i UCAN CRC sedan 2014.,True,2023-08-18,FEMALE,"HEMA, LC, CRC",2021-09-24,AASTADIUM4,"C82.9 - Follikulärt lymfom, ospecificerat",Benmärg + blod,NaN,NaN,NaN,IMMUNOTHERAPY,REMISSION,2021-10-28,2022-02-21,Rituximab,2021-09-24,BIOPSY,BONE_MARROW,NaN,Diagnos: FL,2021-10-13,BIOPSY,OTHER,Thyroideabiopsi,FL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1279,npr988864921432,195612,1956,12,2021.0,GAVLE,GAVLEBORG,2021-08-31,NaN,NaN,False,NaN,FEMALE,"LC, HEMA",2021-08-16,NaN,C83.3 - Diffust storcelligt B-cellslymfom,har även adenocarcinom hö lunga.,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2021-09-03,2021-12-21,R-CHOEP14x6 + R-Cytosar samt MTX i högdos.,2021-07-26,BIOPSY,OTHER,hö mellanlob,adenocarcinom,2021-08-16,BIOPSY,OTHER,lilla bäckenet,lymfom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1282,npr995000010364,195312,1953,12,2019.0,UPPSALA,DALARNA,2019-04-24,NaN,NaN,False,NaN,MALE,HEMA,NaN,AASTADIUM4,C83.1 - Mantelcellslymfom,Diagnos i Falun,NaN,NaN,NaN,CHEMOTHERAPY,CURATIVE,2019-05-14,2019-08-31,R-CHOP/R-DHAX,2019-04-24,BIOPSY,BONE_MARROW,Na

In [75]:
temp = clinical_data.groupby('personReference').agg({'consentDate':'count'}).reset_index()
temp[temp.consentDate>1]

,personReference,consentDate


### Testing npr column values

In [79]:
npr_notin_clinicaldata = exams_npr_set.difference(clinical_npr_set)
print(npr_notin_clinicaldata)

{'npr959703731992', 'npr467134513124', 'npr344930616506', 'npr261612855696', 'npr421064151804', 'npr444859213901', 'npr882266785335', 'npr809610189287', 'npr437726777984', 'npr679961010342', 'npr196288896583', 'npr788802379464', 'npr517273193557', 'npr353131040011', 'npr981339069173', 'npr151462634697', 'npr874401858520', 'npr291281941375', 'npr526794682703', 'npr428707591273', 'npr323296159264', 'npr413511988429', 'npr720175180360', 'npr170667868768', 'npr253690274264', 'npr456413476486', 'npr696287093420', 'npr750785342192', 'npr274962080070', 'npr801182059852', 'npr312833307758', 'npr671684118786', 'npr383171534253', 'npr556496837355', 'npr187395319783', 'npr146743887112', 'npr472289465443', 'npr530256734131', 'npr986314225841', 'npr375870041895', 'npr948124918404', 'npr746398964897', 'npr414552343343', 'npr802055489476', 'npr537544941916', 'npr732570862056', 'npr400132717445', 'npr717322658733', 'npr890936583478', 'npr678348115123', 'npr138028683671', 'npr846021868600', 'npr7069176

In [146]:
npr_missingin_clinicaldata = pd.DataFrame(npr_notin_clinicaldata, columns=['npr'])
npr_missingin_clinicaldata.to_excel("/home/ashish/Ashish/UCAN/npr_missingin_clinicaldata_export_20231129.xlsx", index=False)

In [156]:
npr_missingin_clinicaldata.npr.nunique()

705

In [172]:
705+371

1076

In [158]:
len(exams_npr_set)

1076

In [80]:
for npr in npr_notin_clinicaldata:
    for npr2 in clinical_npr_list:
        if npr in npr2:
            print(npr)

In [84]:
for npr in clinical_npr_list:
    for i in npr:
        if i.isalpha() or i.isnumeric():
            pass
        else:
            print(npr)

In [138]:
previous_linked = pd.read_excel('/home/ashish/Ashish/UCAN/linked_data_390patientmatches_clinicaldata_export_211021.xlsx')
previous_linked = previous_linked[["npr", "includingHospital"]]

previous_linked.npr.nunique()

390

In [142]:
new_linked = pd.read_excel('/home/ashish/Ashish/UCAN/linked_data_371patientmatches_clinicaldata_export_20231129.xlsx')
new_linked = new_linked[["npr", "includingHospital"]]
new_linked.npr.nunique()

371

In [143]:
combined = pd.concat([new_linked, previous_linked], ignore_index=True)
combined.npr.nunique()

390

In [144]:
combined

,npr,includingHospital
0,lpr385705046400,UPPSALA
1,lpr415675513429,UPPSALA
2,lpr415675513429,UPPSALA
3,npr100169878450,UPPSALA
4,npr100169878450,UPPSALA
...,...,...
1495,npr997877056847,NaN
1496,npr997877056847,NaN
1497,npr998272047145,NaN
1498,npr999062108241,GAVLE


In [149]:
len(set(previous_linked.npr.to_list()).intersection(set(new_linked.npr.to_list())))

371

In [109]:
df1 = pd.read_excel("/home/ashish/Ashish/UCAN/U-CAN Lymfom - Ahlström 211021 - utlämnat_pnr_removed.xlsx", header=1)
print(df1.shape)
df2 = pd.read_excel("/home/ashish/Ashish/UCAN/20231129_UCAN_lym_export/Lymphoma_Octopus_Export 2023-11-29_C80_only_n1291_incl_YYYY_MM_pnr_removed.xlsx")
print(df2.shape)

(1480, 39)
(1291, 44)


In [110]:
combined_df = pd.concat([df1, df2], ignore_index=True)
print(combined_df.shape)


(2771, 49)


In [111]:
len(np.unique(combined_df.personReference))

1719

In [112]:
print(len(np.unique(df1.personReference)))
print(len(np.unique(df2.personReference)))

1480
1291


In [154]:
len(set(df2.personReference.to_list()).intersection(set(df1.personReference.to_list())))

1052

In [152]:
1291+239

1530